# Imports

In [1]:
# Import utils
import numpy as np
import pandas as pd
import copy
import time
import datetime as dt
import pickle
import json
from pathlib import Path
import joblib
from joblib import dump, load, Parallel, delayed
import os
import itertools
import contextlib
from tqdm import tqdm

from Experiment import Experiment
from Experiment import Evaluation

In [2]:
# Setup the experiment
experiment_setup = dict(

    # Set paths
    path_data = '/home/fesc/DDDInventoryControl/Data',
    path_weightsmodel = '/home/fesc/DDDInventoryControl/Data/WeightsModel/OutOfSample',
    path_results = '/home/fesc/DDDInventoryControl/Data/Results/OutOfSample',
    
    # Weights models
    global_weightsmodel = 'rfwm_global', 
    local_weightsmodel = 'rfwm_local', 

    # Optimization models
    GwSAA = 'GwSAA',
    GwSAAR = 'GwSAAR',
    wSAA = 'wSAA',
    wSAAR = 'wSAAR',
    SAA = 'SAA',
    ExPost = 'ExPost',
    
    # Set product identifiers
    products = range(1,460+1),   # Products (SKUs) k=1,...,M
    
    # Set problem params
    T = 13,             # Planning horizon T
    ts = range(1,13+1), # Periods t=1,...,T of the planning horizon
    taus = [0,1,2,3,4], # Look-aheads tau=0,...,4
    es = [0.33, 0.67, 1.00, 1.33, 1.67, 2.00, 2.33, 2.67, 3.00], # Uncertainty set specifications e
           
    # Set cost params
    cost_params = [
        {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}
    ]
)

# Make all experiment variables visible locally
locals().update(experiment_setup)

# Initialize Evaluation
experiment = Experiment(**experiment_setup)
evaluation = Evaluation(**experiment_setup)

# I. Prescriptive performance results

## Concatenate and aggregate results per model

In [3]:
# Rolling Horizon Global Weighted SAA (GwSAA)
results = evaluation.concatenate_results(path_results+'/'+GwSAA, GwSAA, products, taus)
results.to_csv(path_results+'/'+GwSAA+'_results.csv', sep=',', index=False)
results_agg = evaluation.aggregate_results(results, ['CR', 'tau', 'product'])
results_agg.to_csv(path_results+'/'+GwSAA+'_results_summary.csv', sep=',', index=False)

In [4]:
# Rolling Horizon Local Weighted SAA (wSAA)
results = evaluation.concatenate_results(path_results+'/'+wSAA, wSAA, products, taus)
results.to_csv(path_results+'/'+wSAA+'_results.csv', sep=',', index=False)
results_agg = evaluation.aggregate_results(results, ['CR', 'tau', 'product'])
results_agg.to_csv(path_results+'/'+wSAA+'_results_summary.csv', sep=',', index=False)

In [5]:
# Rolling Horizon Global Robust Weighted SAA (GwSAA-R)
with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(es))) as progress_bar:
    results_ = Parallel(n_jobs=len(es))(delayed(evaluation.concatenate_results)(
        path_results+'/'+GwSAAR, GwSAAR, products, taus, [es_]) for es_ in es)
    
results = pd.concat(results_)
results.to_csv(path_results+'/'+GwSAAR+'_results.csv', sep=',', index=False)
results_agg = evaluation.aggregate_results(results, ['CR', 'tau', 'e', 'product'])
results_agg.to_csv(path_results+'/'+GwSAAR+'_results_summary.csv', sep=',', index=False)

Progress: 100%|██████████| 9/9 [00:11<00:00,  1.24s/it]


In [6]:
# Rolling Horizon Local Robust Weighted SAA (wSAA-R)
with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(es))) as progress_bar:
    results_ = Parallel(n_jobs=len(es))(delayed(evaluation.concatenate_results)(
        path_results+'/'+wSAAR, wSAAR, products, taus, [es_]) for es_ in es)
    
results = pd.concat(results_)
results.to_csv(path_results+'/'+wSAAR+'_results.csv', sep=',', index=False)
results_agg = evaluation.aggregate_results(results, ['CR', 'tau', 'e', 'product'])
results_agg.to_csv(path_results+'/'+wSAAR+'_results_summary.csv', sep=',', index=False)

Progress: 100%|██████████| 9/9 [00:08<00:00,  1.01it/s]


In [7]:
# Rolling Horizon Local SAA (SAA)
results = evaluation.concatenate_results(path_results+'/'+SAA, SAA, products, taus)
results.to_csv(path_results+'/'+SAA+'_results.csv', sep=',', index=False)
results_agg = evaluation.aggregate_results(results, ['CR', 'tau', 'product'])
results_agg.to_csv(path_results+'/'+SAA+'_results_summary.csv', sep=',', index=False)

In [8]:
# Ex-post Clairvoyant (ExPost)
results = evaluation.concatenate_results(path_results+'/'+ExPost, ExPost, products, taus=[None])
results.to_csv(path_results+'/'+ExPost+'_results.csv', sep=',', index=False)
results_agg = evaluation.aggregate_results(results, ['CR', 'product'])
results_agg.to_csv(path_results+'/'+ExPost+'_results_summary.csv', sep=',', index=False)

In [9]:
# Load aggregated results
results_GwSAA = pd.read_csv(path_results+'/'+GwSAA+'_results_summary.csv')
results_wSAA = pd.read_csv(path_results+'/'+wSAA+'_results_summary.csv')
results_GwSAAR = pd.read_csv(path_results+'/'+GwSAAR+'_results_summary.csv')
results_wSAAR = pd.read_csv(path_results+'/'+wSAAR+'_results_summary.csv')
results_SAA = pd.read_csv(path_results+'/'+SAA+'_results_summary.csv')
results_ExPost = pd.read_csv(path_results+'/'+ExPost+'_results_summary.csv')

In [10]:
# Combine to one results data set
cols = ['model', 'CR', 'tau', 'e', 'product', 'K', 'u', 'h', 'b', 
        'I', 'q', 'I_q', 'y', 'I_q_y', 'n_stockouts', 'n_periods', 'c_o', 'c_s', 'cost',
       'defaulted', 'solutions', 'gap', 'exec_time_sec', 'cpu_time_sec']

results_GwSAA['model'] = copy.deepcopy(GwSAA)
results_GwSAA['e'] = None
results_GwSAA = results_GwSAA[cols]

results_wSAA['model'] = copy.deepcopy(wSAA)
results_wSAA['e'] = None
results_wSAA = results_wSAA[cols]

results_GwSAAR['model'] = copy.deepcopy(GwSAAR)
results_GwSAAR = results_GwSAAR[cols]

results_wSAAR['model'] = copy.deepcopy(wSAAR) 
results_wSAAR = results_wSAAR[cols]

results = pd.concat([results_GwSAA, results_wSAA, results_GwSAAR, results_wSAAR])

In [11]:
# Add SAA and ExPost
results = pd.merge(left=results,
                   right=results_SAA[['CR', 'product', 'cost', 'n_stockouts']],
                   on=['CR', 'product'],
                   suffixes=('', '_SAA'))
results = pd.merge(left=results,
                   right=results_ExPost[['CR', 'product', 'cost', 'n_stockouts']],
                   on=['CR', 'product'],
                   suffixes=('', '_ExPost'))

In [12]:
# Save
results.to_csv(path_results+'/Results_summary.csv', sep=',', index=False)

## Select model parameters ($\tau$ and $e$)

In [13]:
# Load aggregated results
results_GwSAA = pd.read_csv(path_results+'/'+GwSAA+'_results_summary.csv')
results_wSAA = pd.read_csv(path_results+'/'+wSAA+'_results_summary.csv')
results_GwSAAR = pd.read_csv(path_results+'/'+GwSAAR+'_results_summary.csv')
results_wSAAR = pd.read_csv(path_results+'/'+wSAAR+'_results_summary.csv')
results_SAA = pd.read_csv(path_results+'/'+SAA+'_results_summary.csv')
results_ExPost = pd.read_csv(path_results+'/'+ExPost+'_results_summary.csv')

In [14]:
# Find best tau
results_GwSAA_best = evaluation.best_tau(results_GwSAA, results_ExPost, groupby=['CR', 'product'])
results_wSAA_best = evaluation.best_tau(results_wSAA, results_ExPost, groupby=['CR', 'product'])
results_GwSAAR_best = evaluation.best_tau(results_GwSAAR, results_ExPost, groupby=['CR', 'product', 'e'])
results_wSAAR_best = evaluation.best_tau(results_wSAAR, results_ExPost, groupby=['CR', 'product', 'e'])
results_SAA_best = evaluation.best_tau(results_SAA, results_ExPost, groupby=['CR', 'product'])

In [15]:
# Find best e given best tau for robust models
results_GwSAAR_best = evaluation.best_e(results_GwSAAR_best, results_ExPost, groupby=['CR', 'product'])
results_wSAAR_best = evaluation.best_e(results_wSAAR_best, results_ExPost, groupby=['CR', 'product'])

In [16]:
# Save
results_GwSAA_best.to_csv(path_results+'/'+GwSAA+'_results_summary_best.csv', sep=',', index=False)
results_wSAA_best.to_csv(path_results+'/'+wSAA+'_results_summary_best.csv', sep=',', index=False)
results_GwSAAR_best.to_csv(path_results+'/'+GwSAAR+'_results_summary_best.csv', sep=',', index=False)
results_wSAAR_best.to_csv(path_results+'/'+wSAAR+'_results_summary_best.csv', sep=',', index=False)
results_SAA_best.to_csv(path_results+'/'+SAA+'_results_summary_best.csv', sep=',', index=False)

In [17]:
# Combine to one results data set
cols = ['model', 'CR', 'tau', 'e', 'product', 'K', 'u', 'h', 'b', 
        'I', 'q', 'I_q', 'y', 'I_q_y', 'n_stockouts', 'n_periods', 'c_o', 'c_s', 'cost',
       'defaulted', 'solutions', 'gap', 'exec_time_sec', 'cpu_time_sec']

results_GwSAA_best['model'] = copy.deepcopy(GwSAA)
results_GwSAA_best['e'] = None
results_GwSAA_best = results_GwSAA_best[cols]

results_wSAA_best['model'] = copy.deepcopy(wSAA)
results_wSAA_best['e'] = None
results_wSAA_best = results_wSAA_best[cols]

results_GwSAAR_best['model'] = copy.deepcopy(GwSAAR)
results_GwSAAR_best = results_GwSAAR_best[cols]

results_wSAAR_best['model'] = copy.deepcopy(wSAAR) 
results_wSAAR_best = results_wSAAR_best[cols]

results = pd.concat([results_GwSAA_best, results_wSAA_best, results_GwSAAR_best, results_wSAAR_best])

In [18]:
# Add SAA and ExPost
results = pd.merge(left=results,
                   right=results_SAA_best[['CR', 'product', 'cost', 'n_stockouts']],
                   on=['CR', 'product'],
                   suffixes=('', '_SAA'))
results = pd.merge(left=results,
                   right=results_ExPost[['CR', 'product', 'cost', 'n_stockouts']],
                   on=['CR', 'product'],
                   suffixes=('', '_ExPost'))

In [19]:
# Save
results.to_csv(path_results+'/Results_summary_best.csv', sep=',', index=False)

# II. Predictive performance results

In [20]:
n_jobs=32

In [21]:
# Initialize
results_global = {}
results_local = {}

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Status
    print('#### Look-ahead tau='+str(tau)+'...')

    # Initialize Experiment
    experiment = Experiment(**experiment_setup)
    evaluation = Evaluation(**experiment_setup)
    
    # Initialize
    results_global[tau] = {}
    results_local[tau] = {}

    # Load preprocessed data (alternatively, data can be preprocessed here)
    data_global = load(path_weightsmodel+'/global_data_tau'+str(tau)+'.joblib')
    data_local = load(path_weightsmodel+'/local_data_tau'+str(tau)+'.joblib')

    # Load weights
    weights_global = load(path_weightsmodel+'/'+global_weightsmodel+'_weights_tau'+str(tau)+'.joblib') 
    weights_local = load(path_weightsmodel+'/'+local_weightsmodel+'_weights_tau'+str(tau)+'.joblib') 

    # Preprocess experiment data
    weights_global_, samples_global_, actuals_global_ = experiment.preprocess_data(data_global, weights_global)
    weights_local_, samples_local_, actuals_local_ = experiment.preprocess_data(data_local, weights_local)

    # Evaluation of predictive performance: global training and sampling
    with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
        results = Parallel(n_jobs=n_jobs)(delayed(evaluation.predictive_performance)(weights=weights_global_[product], 
                                                                                     samples=samples_global_[product], 
                                                                                     samples_saa=samples_local_[product], 
                                                                                     actuals=actuals_local_[product]) for product in products)
        
        
    for p in range(len(results)):
        results_global[tau][products[p]] = results[p]

    # Evaluation of predictive performance: local training and sampling
    with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
        results = Parallel(n_jobs=n_jobs)(delayed(evaluation.predictive_performance)(weights=weights_local_[product], 
                                                                                     samples=samples_local_[product], 
                                                                                     samples_saa=samples_local_[product], 
                                                                                     actuals=actuals_local_[product]) for product in products)

    for p in range(len(results)):
        results_local[tau][products[p]] = results[p]

#### Look-ahead tau=0...


Progress: 100%|██████████| 460/460 [00:03<00:00, 125.14it/s]


#### Look-ahead tau=1...


Progress: 100%|██████████| 460/460 [00:06<00:00, 68.74it/s]


#### Look-ahead tau=2...


Progress: 100%|██████████| 460/460 [00:08<00:00, 51.54it/s]


#### Look-ahead tau=3...


Progress: 100%|██████████| 460/460 [00:11<00:00, 40.75it/s]


#### Look-ahead tau=4...


Progress: 100%|██████████| 460/460 [00:13<00:00, 34.27it/s]


In [22]:
# Post-processing 
results_rmsse_global, results_rmsse_local = [], []
results_spl_global, results_spl_local = [], []
   
# For each look-ahead tau=0,...,5
for tau in taus:
    
    result_rmsse_global, result_rmsse_local = [], []
    result_spl_global, result_spl_local = [], []

    # For each product (SKU) k=1,...,M
    for product in products:
        
        # Get results
        rmsse_global, spl_global = results_global[tau][product]
        rmsse_local, spl_local = results_local[tau][product]
        
        # Append over products
        result_rmsse_global += [rmsse_global]
        result_spl_global += [pd.DataFrame({'tau': tau, 'product': product, 'u': spl_global.keys(), 'spl': spl_global.values()})]
 
        result_rmsse_local += [rmsse_local]
        result_spl_local += [pd.DataFrame({'tau': tau, 'product': product, 'u': spl_local.keys(), 'spl': spl_local.values()})]

    # Append over tau
    results_rmsse_global += [pd.DataFrame({'tau': tau, 'product': products, 'rmsse': result_rmsse_global})]
    results_spl_global += [pd.concat(result_spl_global).reset_index(drop=True)]

    results_rmsse_local += [pd.DataFrame({'tau': tau, 'product': products, 'rmsse': result_rmsse_local})]
    results_spl_local += [pd.concat(result_spl_local).reset_index(drop=True)]

# Finalize
results_rmsse_global = pd.concat(results_rmsse_global).reset_index(drop=True)
results_spl_global = pd.concat(results_spl_global).reset_index(drop=True)

results_rmsse_local = pd.concat(results_rmsse_local).reset_index(drop=True)
results_spl_local = pd.concat(results_spl_local).reset_index(drop=True)

In [23]:
# Save predictive performance results
results_rmsse_global.to_csv(path_results+'/Predictive_performance_rmsse_global.csv', sep=',', index=False)
results_spl_global.to_csv(path_results+'/Predictive_performance_spl_global.csv', sep=',', index=False)

results_rmsse_local.to_csv(path_results+'/Predictive_performance_rmsse_local.csv', sep=',', index=False)
results_spl_local.to_csv(path_results+'/Predictive_performance_spl_local.csv', sep=',', index=False)

/home/fesc/DDDInventoryControl/Code/Experiment.py:826: RuntimeWarning: invalid value encountered in double_scalars
  spl = (pl == pl_saa) * 1.0 + (pl != pl_saa) * (pl / pl_saa)
/home/fesc/DDDInventoryControl/Code/Experiment.py:826: RuntimeWarning: invalid value encountered in double_scalars
  spl = (pl == pl_saa) * 1.0 + (pl != pl_saa) * (pl / pl_saa)
/home/fesc/DDDInventoryControl/Code/Experiment.py:826: RuntimeWarning: invalid value encountered in double_scalars
  spl = (pl == pl_saa) * 1.0 + (pl != pl_saa) * (pl / pl_saa)
/home/fesc/DDDInventoryControl/Code/Experiment.py:826: RuntimeWarning: invalid value encountered in double_scalars
  spl = (pl == pl_saa) * 1.0 + (pl != pl_saa) * (pl / pl_saa)
/home/fesc/DDDInventoryControl/Code/Experiment.py:826: RuntimeWarning: invalid value encountered in double_scalars
  spl = (pl == pl_saa) * 1.0 + (pl != pl_saa) * (pl / pl_saa)
/home/fesc/DDDInventoryControl/Code/Experiment.py:826: RuntimeWarning: invalid value encountered in double_scalars
